In [ ]:
# Import the nessecary libraries
from __future__ import print_function
import argparse
import os
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter
import numpy as np

In [ ]:
class ConvNet(nn.Module):
  # The initializer or Constructor
  def __init__(self, mode):
    super(ConvNet, self).__init__()

    # Define various layers here
    self.fc1 = nn.Linear(28*28, 18)
    self.fc2 = nn.Linear(18, 15)
    self.fc3 = nn.Linear(15, 10)

    self.deepfc1 = nn.Linear(28*28, 250)
    self.deepfc2 = nn.Linear(250, 250)
    self.deepfc3 = nn.Linear(250, 250)
    self.deepfc4 = nn.Linear(250, 230)
    self.deepfc5 = nn.Linear(230, 230)
    self.deepfc6 = nn.Linear(230, 10)

    # This will select the forward pass function based on mode for the ConvNet.
    # During the creation of each ConvNet model, you will assign one of the valid modes.
    # This will fix the forward function (and the network graph) for the entire
    # training/testing
    if mode == 1:
      self.forward = self.model_1
    elif mode == 2:
      self.forward = self.model_2
    elif mode == 3:
      self.forward = self.model_3
    else:
      print('Invalid mode {} selected. Select between 1-3'.format(mode))
      exit(0)

  # Baseline sample model
  def model_0(self, X):
    # ======================================================================
    # Three fully connected layers with activation

    X = torch.flatten(X, start_dim=1)
    X = self.fc1(X)
    X = F.relu(X)
    X = self.fc2(X)
    X = F.relu(X)
    X = self.fc3(X)
    X = torch.sigmoid(X)

    return X

  # Base line model. Task 1
  def model_1(self, X):
    # ======================================================================
    # Three fully connected layers without activation

    X = torch.flatten(X, start_dim=1)
    X = self.fc1(X)
    X = self.fc2(X)
    X = self.fc3(X)

    return X

  # Task 2
  def model_2(self, X):
    # ======================================================================
    # Train with activation (use model 1 from task 1)

    X = torch.flatten(X, start_dim=1)
    X = self.fc1(X)
    X = F.relu(X)
    X = self.fc2(X)
    X = F.relu(X)
    X = self.fc3(X)
    X = torch.sigmoid(X)

    return X

  # Task 3
  def model_3(self, X):
    # ======================================================================
    # Change number of fully connected layers and number neurons from model
    # 2 in task 2

    X = torch.flatten(X, start_dim=1)
    X = F.relu(self.deepfc1(X))
    X = F.relu(self.deepfc2(X))
    X = F.relu(self.deepfc3(X))
    X = F.relu(self.deepfc4(X))
    X = F.relu(self.deepfc5(X))
    X = F.relu(self.deepfc6(X))
    X = torch.sigmoid(X)

    return X

In [ ]:
def train(model, device, train_loader, optimizer, criterion, epoch, batch_size):
  '''
  Trains the model for an epoch and optimizes it.
  model: The model to train. Should already be in correct device.
  device: 'cuda' or 'cpu'.
  train_loader: dataloader for training samples.
  optimizer: optimizer to use for model parameter updates.
  criterion: used to compute loss for prediction and target 
  epoch: Current epoch to train for.
  batch_size: Batch size to be used.
  '''
  
  # Set model to train mode before each epoch
  model.train()

  # Empty list to store losses
  losses = []
  correct = 0
  
  # Iterate over entire training samples (1 epoch)
  for batch_idx, batch_sample in enumerate(train_loader):
    data, target = batch_sample

    # Push data/label to correct device
    data, target = data.to(device), target.to(device)
    
    # Reset optimizer gradients. Avoids grad accumulation (accumulation used in RNN)
    optimizer.zero_grad()
    
    # Do forward pass for current set of data
    output = model(data)
    
    # ======================================================================
    # Compute loss based on criterion
    # ----------------- YOUR CODE HERE ----------------------
    #
    # Remove NotImplementedError and assign correct loss function.
    # loss = NotImplementedError()
    loss = criterion(output, target)
    
    # Computes gradient based on final loss
    loss.backward()

    # Store loss
    losses.append(loss.item())

    # Optimize model parameters based on learning rate and gradient
    optimizer.step()

    # Get predicted index by selecting maximum log-probability
    pred = output.argmax(dim=1, keepdim=True)

    # ======================================================================
    # Count correct predictions overall
    correct += pred.eq(target.view_as(pred)).sum().item()
  
  train_loss = float(np.mean(losses))
  train_acc = correct / ((batch_idx+1) * batch_size)

  print('Train set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        float(np.mean(losses)), correct, (batch_idx+1) * batch_size,
        100. * correct / ((batch_idx+1) * batch_size)))

  return train_loss, train_acc

In [ ]:
def test(model, device, test_loader):
  '''
  Tests the model.
  model: The model to train. Should already be in correct device.
  device: 'cuda' or 'cpu'.
  test_loader: dataloader for test samples.
  '''

  # Set model to eval mode to notify all layers
  model.eval()

  losses = []
  correct = 0

  # Set torch.no_grad() to disable gradient computation and backpropagation
  with torch.no_grad():
    for batch_idx, sample in enumerate(test_loader):
      data, target = sample
      data, target = data.to(device), target.to(device)

      # Predict for data by doing forward pass
      output = model(data)

      # ======================================================================
      # Compute loss based on same criterion as training
      loss = F.cross_entropy(output, target, reduction='mean')

      # Append loss to overall test loss
      losses.append(loss.item())

      # Get predicted index by selecting maximum log-probability
      pred = output.argmax(dim=1, keepdim=True)

      # ======================================================================
      # Count correct predictions overall
      correct += pred.eq(target.view_as(pred)).sum().item()

  test_loss = float(np.mean(losses))
  accuracy = 100. * correct / len(test_loader.dataset)

  print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset), accuracy))

  return test_loss, accuracy

In [ ]:
def run_main(FLAGS):
  # Check if cuda is available
  use_cuda = torch.cuda.is_available()
  
  # Set proper devide based on cuda availability
  device = torch.device("cuda" if use_cuda else "cpu")
  print("Torch device selected: ", device)

  #Initialize the model and send to device
  model = ConvNet(FLAGS.mode).to(device)
  
  # Initialize the criterion for loss computation 
  # ======================================================================
  # Remove NotImplementedError and assign correct loss function.
  # criterion = NotImplementedError()
  criterion = nn.CrossEntropyLoss(reduction='mean')

  # Initialize optimizer type
  optimizer = optim.SGD(model.parameters(), lr=FLAGS.learning_rate, weight_decay=1e-7)

  # Create transformations to apply to each data sample
  # Can specify variations such as image flip, color flip, random crop, ...
  transform = transforms.Compose([
      transforms.ToTensor(),
      transforms.Normalize((0.1307,), (0.3081,))
  ])

  # Load datasets for training and testing
  # Inbuild datasets available in torchvision (check documentation online)
  dataset1 = datasets.MNIST('./data/', train=True, download=True, transform=transform)
  dataset2 = datasets.MNIST('./data/', train=False, transform=transform)
  train_loader = DataLoader(dataset1, batch_size=FLAGS.batch_size,
                            shuffle=True, num_workers=4)
  test_loader = DataLoader(dataset2, batch_size=FLAGS.batch_size,
                           shuffle=False, num_workers=4)
  
  best_accuracy = 0.0

  # Run training for n_epochs specified in config
  for epoch in range(1, FLAGS.num_epochs + 1):
    print("\nEpoch: ", epoch)
    train_loss, train_accuracy = train(model, device, train_loader,
                                       optimizer, criterion, epoch, FLAGS.batch_size)
    test_loss, test_accuracy = test(model, device, test_loader)

    if test_accuracy > best_accuracy:
      best_accuracy = test_accuracy

  print('\naccuracy is {:2.2f}'.format(best_accuracy))

  print('Training and evaluation finished')

In [ ]:
# Set parameters for Sparse Autoencoder
parser = argparse.ArgumentParser('CNN Exercise.')
parser.add_argument('--mode', type=int, default=1,
                    help='Select mode between 1-3')
parser.add_argument('--learning_rate', type=float, default=0.01,
                    help='Initial learning rate')
parser.add_argument('--num_epochs', type=int, default=20,
                    help='Number of epochs to run trainer')
parser.add_argument('--batch_size', type=int, default=10,
                    help='Batch size. Must divide evenly into the dataset sizes.')
parser.add_argument('--log_dir', type=str, default='logs',
                    help='Directory to put logging')

FLAGS = None
FLAGS, unparsed = parser.parse_known_args()

print("Mode: {}".format(FLAGS.mode))
print("LR: {}".format(FLAGS.learning_rate))
print("Batch size: {}".format(FLAGS.batch_size))

run_main(FLAGS)

Mode: 1
LR: 0.01
Batch size: 10
Torch device selected:  cuda

Epoch:  1


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Train set: Average loss: 1.7157, Accuracy: 45425/60000 (76%)
Test set: Average loss: 1.6060, Accuracy: 8146/10000 (81%)

Epoch:  2
Train set: Average loss: 1.6003, Accuracy: 48795/60000 (81%)
Test set: Average loss: 1.5863, Accuracy: 8214/10000 (82%)

Epoch:  3
Train set: Average loss: 1.5859, Accuracy: 49744/60000 (83%)
Test set: Average loss: 1.5762, Accuracy: 8372/10000 (84%)

Epoch:  4
Train set: Average loss: 1.5777, Accuracy: 50830/60000 (85%)
Test set: Average loss: 1.5722, Accuracy: 8569/10000 (86%)

Epoch:  5
Train set: Average loss: 1.5728, Accuracy: 51620/60000 (86%)
Test set: Average loss: 1.5669, Accuracy: 8836/10000 (88%)

Epoch:  6
Train set: Average loss: 1.5690, Accuracy: 52395/60000 (87%)
Test set: Average loss: 1.5644, Accuracy: 8744/10000 (87%)

Epoch:  7
Train set: Average loss: 1.5649, Accuracy: 53261/60000 (89%)
Test set: Average loss: 1.5609, Accuracy: 8984/10000 (90%)

Epoch:  8
Train set: Average loss: 1.5605, Accuracy: 53873/60000 (90%)
Test set: Average loss